# tunelyze Interactive
Analyze Your Tunes™

Uses the `tunelyze` command-line toolsuite to fetch songs from your saved music library, saved albums, and playlists, output/analyze/graph the data, and generate playlists from the source music with feature filters.

## Setup

In [ ]:
import sys
sys.path.append('../')

import api.spotify as spotify
import api.utils as utils

from api.spotify import FeatureType, FeatureFilter

import clt.input_tools as clinput
import clt.tunelyze as tunelyze
import clt.visualizer as viz

First, we need to load the application data and username, and login.

In [ ]:
username, scope = tunelyze.boot('config.yml')
with spotify.login(username, scope) as sp_login:
    sp = sp_login

## Music Sources
Now, you'll need to choose where you want the source music to come from. You can either specify these using the picker below, which allows you choose individual playlists, indivudual albums (from the ones you've saved to your library), and whether or not to use all the songs you individually saved to your library.

In [ ]:
# Music source picker
use_saved_tracks = clinput.input_boolean("Do you want me to analyze your saved songs?")
print()
print("Saved Albums:")
queried_albums = clinput.input_sublist("Which albums would you like to include in the analysis?", sp.get_saved_albums())
print()
print("Playlists:")
queried_playlists = clinput.input_sublist("Which playlists would you like to include in the analysis?", sp.get_playlists())

print("="*12)
print("including songs from:")
if use_saved_tracks:
    print("  - your Saved Music library")
if len(queried_albums) != 0:
    print("  - the following albums: {}".format(utils.get_english_list(queried_albums)))
if len(queried_playlists) != 0:
    print("  - the following playlists: {}".format(utils.get_english_list(queried_playlists)))

Now, we can pull all tracks from those sources and get the features, courtesy of the Spotify API.

In [ ]:
print("Fetching songs...")
tracks = sp.get_tracks(use_saved_tracks, [album.id for album in queried_albums], [playlist.id for playlist in queried_playlists])
print("   Done.")

print("Getting features...")
sp.load_features(tracks)
print("   Done.")

If you'd like, you can print out all of the tracks, their artists, and their features in a delimeted spreadsheet format. Unfortunaely, because of all of the commas present in titles and artists, you might not be able to use a CSV-format, so specify the delimeter to your liking.

In [ ]:
delimiter = '---'
for line in tunelyze.get_delimited_spreadsheet(tracks, delimiter):
    print(line)

## Feature Visualization
Because we have a lot of numbers, we can make fun histograms and pie charts and bar graphs for all of the music you selected.

The actual definition of each of the features can be found on [Spotify's API docs, linked here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/#audio-features-object).

In [ ]:
%matplotlib inline
viz.plot_all_features(tracks)

## Feature Filtering
In addition to just looking at the distribution of features for the songs, we can also get subsets of these songs based on `FeatureFilter`s, which are defined on ranges (for the numerical features) or lists of values (for the discrete-valued filters of key and mode).

In addition to the `Track.satisfies_all(filters)` and `Track.satsisfies_any(filters)`, you can also use `FeatureFilter.is_satisfied_by(track)` to make more complex filter predicates.

In [ ]:
# This predicate attempts to make a lower-energy playlist.
filters = [
    FeatureFilter(FeatureType.ENERGY,       min_val=   0.0, max_val=   0.6),
    FeatureFilter(FeatureType.TEMPO,        min_val=   0.0, max_val= 130.0),
    FeatureFilter(FeatureType.LOUDNESS,     min_val= -60.0, max_val=  -7.0)
]

predicate = lambda track : track.satisfies_all(filters)

In [ ]:
# This predicate attempts to make a dance-y playlist
filters = [
    FeatureFilter(FeatureType.ENERGY,         min_val=    0.6,  max_val=    1.0),
    FeatureFilter(FeatureType.DANCEABILITY,   min_val=    0.65, max_val=    1.0)
]

predicate = lambda track : track.satisfies_all(filters)

In [ ]:
# This predicare only allows music in the key of C minor or E major, if you're in to that...
c_minor = [
    FeatureFilter(FeatureType.KEY,         values=['C']),
    FeatureFilter(FeatureType.MODE,        values=['minor'])
]

e_major = [
    FeatureFilter(FeatureType.KEY,         values=['E']),
    FeatureFilter(FeatureType.MODE,        values=['major'])
]

predicate = lambda track : track.satisfies_all(c_minor) or track.satisfies_all(e_major)

Now, we can get the tracks that satisfy our predicate...

In [ ]:
filtered_tracks = spotify.get_tracks_that_satisfy_predicate(tracks, predicate)

visualize them with respect to our source songs...

In [ ]:
viz.plot_all_features(tracks, filtered_tracks)

and create a playlist with those songs.

In [ ]:
plist_id = sp.create_playlist("dance dance", [track.id for track in filtered_tracks])
"Playlist URL: https://open.spotify.com/playlist/" + plist_id